In [1]:
import matplotlib.pyplot as plt
import pickle
import itertools
import pandas as pd
import numpy as np
import scipy.stats as st
import random 
from scipy.optimize import curve_fit
import math

%matplotlib inline 

In [2]:
def translateDNA(seq):
    i = 0
    aa = ""
    while i <= len(seq)-3:
        aa += geneticcode[seq[i:i+3].lower()]
        i +=3
    return aa

def rev_comp(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A','a': 't', 'c': 'g', 'g': 'c', 't': 'a'}
    return "".join(complement.get(base, base) for base in reversed(seq))


def readSpecRes(fastQfile,n_Ns, print_stats = False):
    fastq1 = open(fastQfile+"_1_sequence.fastq",'r')
    fastq2 = open(fastQfile+"_2_sequence.fastq",'r')
    fifthlines1  = itertools.islice(fastq1,1,None,4)
    fifthlines2  = itertools.islice(fastq2,1,None,4)
    
    #offset
    of = n_Ns - 5
    

    #read HK specificity residues
    hk_template = "XXXXXctgacccatagtctgaaaacgccactgGCGgtgctgcaaAGTACGctgcgtTCTctgcgtagtgaaaagatgagcgtcagtGATGCT"
    hk_template = hk_template.upper()
    h1 = hk_template[6:32]
    h2 = hk_template[35:44]
    h3 = hk_template[50:55]
    h4 = hk_template[59:86]
    
    count = 0
    
    hk_srs = []
    for line in fifthlines1:
        line = line.rstrip()

        if line[6+of:32+of] == h1 and line[35+of:44+of] == h2 and line[50+of:55+of]==h3 and h4 == line[59+of:86+of]:
            SR = line[32+of:35+of] + line[44+of:50+of] + line[56+of:59+of] + line[86+of:92+of]
            hk_srs += [SR]
        else:
            hk_srs += [False]
        
    #read RR specificity residues
    rr_template = "XXXXXcgacctgatgaccagcatcctgaatCTGAACtttaagGTGGTGacgTAAcaacgcattgtcttcaacaaccagtacgcgcat"
    rr_template = rr_template.upper()
    r1 = rr_template[25:30]
    r2 = rr_template[36:42]
    r3 = rr_template[48:51]
    r4 = rr_template[54:58]
    
    count = 0
    rr_srs = []
    for line in fifthlines2:
        line = line.rstrip()
        
        #check if conserved residues
        if line[25+of:30+of]== r1 and line[36+of:42+of] == r2 and line[48+of:51+of]== r3:
            SR = rev_comp(line[30+of:36+of] + line[42+of:48+of]+ line[51+of:54+of])
            rr_srs += [SR]
        else:
            rr_srs += [False]
    
    if len(rr_srs) != len(hk_srs): 
        print "fastQ files for read1 and read2 are different lengths"
        raise ValueError
        
    SR_dict = {}
    errors = 0
    for i in range(len(rr_srs)):
        if rr_srs[i] and hk_srs[i]:
            SR = rr_srs[i]+hk_srs[i]
            try:
                SR_dict[SR] += 1
            except KeyError:
                SR_dict[SR] = 1
            
        else:
            errors += 1
        
    fastq1.close()
    fastq2.close()
    
    if print_stats:
        print "sample", fastQfile[-2:]
        print len(rr_srs), "total reads,", 100*round(1.0-errors/float(len(rr_srs)),4), "% perfect,", len(SR_dict),"unique seqs"
        
    return SR_dict


In [8]:
directory = "../../../../notebooks/storage/mcclune/180412Lau/"

In [15]:
for sID in range(1,59):
    #sample/multiplex ID
    n_Ns = 5
    if sID in  range(37,49):
        n_Ns = 4
    elif sID in  range(49,59):
        n_Ns = 6
        
    if sID < 10:
        fastQfile1 = directory+"D18-13100"+str(sID)+"-3091L/180412Lau_D18-13100"+str(sID) 
    else:
        fastQfile1 = directory+"D18-1310"+str(sID)+"-3091L/180412Lau_D18-1310"+str(sID) 

    
    sr_dict = readSpecRes(fastQfile1, n_Ns, print_stats=True)
    
    sID = sID - 1
    pickle.dump(sr_dict, open("raw_read_dicts/rrd"+str(sID),'w'))

sample 01
5452275 total reads, 74.91 % perfect, 310782 unique seqs
sample 02
8506320 total reads, 75.21 % perfect, 425209 unique seqs
sample 03
9204349 total reads, 75.17 % perfect, 476191 unique seqs
sample 04
7304082 total reads, 74.79 % perfect, 438028 unique seqs
sample 05
5098863 total reads, 76.19 % perfect, 306164 unique seqs
sample 06
829111 total reads, 76.32 % perfect, 69018 unique seqs
sample 07
1922538 total reads, 76.59 % perfect, 142029 unique seqs
sample 08
4934079 total reads, 77.34 % perfect, 288244 unique seqs
sample 09
2017275 total reads, 75.82 % perfect, 115021 unique seqs
sample 10
1665923 total reads, 74.78 % perfect, 125580 unique seqs
sample 11
11710266 total reads, 75.48 % perfect, 617361 unique seqs
sample 12
7887676 total reads, 75.11 % perfect, 428682 unique seqs
sample 13
12261951 total reads, 75.98 % perfect, 649779 unique seqs
sample 14
9824055 total reads, 76.23 % perfect, 592965 unique seqs
sample 15
3032686 total reads, 76.34 % perfect, 205705 unique 